https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/

# Check tensorflow and torch GPU

In [1]:
import tensorflow as tf
print(len(tf.config.list_physical_devices('GPU')))
import torch
print(torch.cuda.is_available())

1
True


# Import important libraries

In [2]:

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from skimage import transform

In [3]:
photo_size=224

# Prepare dataset and add some augmentation

In [13]:
def prepare_dataset(data_dir):
    datagen = ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=40,
        width_shift_range=.2,
        height_shift_range=.2,
        shear_range=.1,
        horizontal_flip=True,
        fill_mode='nearest',
        zoom_range=.2,
    )
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=(photo_size,photo_size),
        class_mode='binary',
        batch_size=128,
        classes=['non_autistic','autistic']
    )
    return generator


# Split training ,validation and  testing Dataset

In [14]:
import shutil
import os
import numpy as np
import argparse

def get_files_from_folder(path):
    files = os.listdir(path)
    return np.asarray(files)

def image_train_test_split(path_to_data, path_to_test_data, train_ratio):
    # get dirs
    _, dirs, _ = next(os.walk(path_to_data))

    # calculates how many train data per class
    data_counter_per_class = np.zeros((len(dirs)))
    for i in range(len(dirs)):
        path = os.path.join(path_to_data, dirs[i])
        files = get_files_from_folder(path)
        data_counter_per_class[i] = len(files)
    test_counter = np.round(data_counter_per_class * (1 - train_ratio))

    # transfers files
    for i in range(len(dirs)):
        path_to_original = os.path.join(path_to_data, dirs[i])
        path_to_save = os.path.join(path_to_test_data, dirs[i])

        #creates dir
        if not os.path.exists(path_to_save):
            os.makedirs(path_to_save)
        files = get_files_from_folder(path_to_original)
        # moves data
        for j in range(int(test_counter[i])):
            dst = os.path.join(path_to_save, files[j])
            src = os.path.join(path_to_original, files[j])
            shutil.move(src, dst)



In [15]:
# image_train_test_split('Train', 'Validation', float(0.9))

# Load Dataset from directory

In [16]:
train_data=prepare_dataset('train/')
validation_data = prepare_dataset('valid/')
test_data=prepare_dataset('test/')


Found 2536 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [17]:
validation_data.class_indices

{'non_autistic': 0, 'autistic': 1}

# Load pre-trained models

## Vgg model

In [32]:
# example of tending the vgg16 model
def create_vgg_model():
    from keras.applications.vgg16 import VGG16
    from keras.models import Model
    from keras.layers import Dense
    from keras.layers import Flatten
    # load model without classifier layers
    model = VGG16(include_top=False, input_shape=(photo_size, photo_size, 3))
    for layer_idx in range(len(model.layers)):
        if layer_idx not in [1,2,3,15,16,17,18]:
            model.layers[layer_idx].trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(256, activation='relu')(flat1)
    output = Dense(95, activation='softmax')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model

In [33]:
vgg_model=create_vgg_model()
vgg_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0   

In [ ]:
vgg_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg_model.fit(
    train_data,
    epochs=50,
    validation_data=validation_data
)
vgg_model.save("vgg_model50.h5")

Epoch 1/50
798/798 [==============================] - 2264s 3s/step - loss: 3.8393 - accuracy: 0.1204 - val_loss: 2.8091 - val_accuracy: 0.2648
Epoch 2/50
798/798 [==============================] - 1229s 2s/step - loss: 1.6265 - accuracy: 0.5493 - val_loss: 1.3089 - val_accuracy: 0.6194
Epoch 3/50
798/798 [==============================] - 1229s 2s/step - loss: 0.7653 - accuracy: 0.7774 - val_loss: 0.9439 - val_accuracy: 0.7355
Epoch 4/50
798/798 [==============================] - 1221s 2s/step - loss: 0.4805 - accuracy: 0.8557 - val_loss: 0.7082 - val_accuracy: 0.8093
Epoch 5/50
798/798 [==============================] - 1223s 2s/step - loss: 0.3628 - accuracy: 0.8928 - val_loss: 0.6459 - val_accuracy: 0.8340
Epoch 6/50
798/798 [==============================] - 1227s 2s/step - loss: 0.3093 - accuracy: 0.9086 - val_loss: 0.6683 - val_accuracy: 0.8280
Epoch 7/50
798/798 [==============================] - 1233s 2s/step - loss: 0.2978 - accuracy: 0.9149 - val_loss: 0.6193 - val_accuracy:

In [ ]:
vgg_model.evaluate(test_data)

## Inception model

In [11]:
def create_inception_model():
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    base_model = InceptionV3(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    # for layer_idx in range(len(pretrained_model.layers)):
    #     if layer_idx not in [1,2,3,305,306,307,308,309,310]:
    #         pretrained_model.layers[layer_idx].trainable = False
    from tensorflow.keras.optimizers import RMSprop
    from tensorflow.keras import layers
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = layers.Dense(95, activation='softmax')(x)
    model = tf.keras.models.Model(base_model.input, x)
    model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model

In [12]:
inception_model=create_inception_model()
inception_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 99, 99, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 99, 99, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                            

In [ ]:
inception_model.fit(
    train_data,
    epochs=50,
    validation_data=validation_data
)
inception_model.save("inception_model.h5")

# Efficient net

In [23]:
def use_efficient_net(model_type='B0'):
    from keras.optimizers import RMSprop
    from keras.models import Model
    import efficientnet.keras as efn
    if model_type=='B0':
        efn_model = efn.EfficientNetB0(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    else:
        efn_model = efn.EfficientNetB7(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    for layer in efn_model.layers:
        layer.trainable = False
    #
    x = efn_model.output
    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    # Add a final sigmoid layer with 1 node for classification output
    predictions = Dense(1, activation="sigmoid")(x)
    efficient_net = Model(efn_model.input,predictions)

    efficient_net.compile(RMSprop(learning_rate=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])
    return efficient_net
efficient_net=use_efficient_net('B0')
efficient_net.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn (BatchNormalization)   (None, 112, 112, 32  128         ['stem_conv[0][0]']              
                                )                                                           

In [24]:
eff_history = efficient_net.fit(train_data, validation_data = validation_data, epochs = 20)
efficient_net.save("autism_efficient_net20.h5")

Epoch 1/20
20/20 [==============================] - 61s 3s/step - loss: 0.7560 - accuracy: 0.6179 - val_loss: 0.7066 - val_accuracy: 0.6100
Epoch 2/20
20/20 [==============================] - 45s 2s/step - loss: 0.5968 - accuracy: 0.6790 - val_loss: 0.5797 - val_accuracy: 0.6700
Epoch 3/20
20/20 [==============================] - 45s 2s/step - loss: 0.5629 - accuracy: 0.7066 - val_loss: 0.6465 - val_accuracy: 0.6700
Epoch 4/20
20/20 [==============================] - 45s 2s/step - loss: 0.5609 - accuracy: 0.7133 - val_loss: 0.5974 - val_accuracy: 0.7200
Epoch 5/20
20/20 [==============================] - 45s 2s/step - loss: 0.5479 - accuracy: 0.7271 - val_loss: 0.6239 - val_accuracy: 0.6400
Epoch 6/20
20/20 [==============================] - 45s 2s/step - loss: 0.5423 - accuracy: 0.7244 - val_loss: 0.5260 - val_accuracy: 0.7300
Epoch 7/20
20/20 [==============================] - 45s 2s/step - loss: 0.5379 - accuracy: 0.7256 - val_loss: 0.4773 - val_accuracy: 0.7600
Epoch 8/20
20/20 [==

In [41]:
efficient_net.evaluate(test_data)

55/55 [==============================] - 657s 12s/step - loss: 0.2100 - accuracy: 0.9507


[0.2100256234407425, 0.9506860971450806]

# Champion model Inference

In [4]:
!pip install efficientnet

In [42]:
from efficientnet.tfkeras import EfficientNetB0
from keras.models import load_model
efficient_net_model= load_model('efficient_net20.h5')
efficient_net_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 100, 100, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn (BatchNormalization)   (None, 100, 100, 32  128         ['stem_conv[0][0]']              
                                )                                                           

In [11]:
# efficient_net_model.evaluate(test_data)

## Load Images from path

In [43]:
from PIL import Image
import numpy as np
from skimage import transform
photo_size=200
def load_image_from_path(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32') / 255
    np_image = transform.resize(np_image, (photo_size, photo_size, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [ ]:
mTestPath = "Hard_tests/"
for test in os.listdir(mTestPath):
    img = load_image_from_path(os.path.join(mTestPath, test))
    res = efficient_net_model.predict(img)[0].argmax()
    print(test, "\t", res)